In [3]:
import importlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from customer_segmentation.constants import feature_groups as fg_const
from customer_segmentation.utils import db_utils, file_io, plot_utils, preprocessing

modules_to_reload = [db_utils, file_io, preprocessing, plot_utils, fg_const]
for module in modules_to_reload:
    importlib.reload(module)

# Set pandas to display all columns
pd.set_option("display.max_columns", None)

In [4]:
cohort = db_utils.read_from_db_to_df(
    sql_filename="customer_features_local", db_mode="local"
)

In [5]:
# convert data types
cohort["user_id"] = cohort["user_id"].astype(str)
cohort["cnt_trips"] = cohort["cnt_trips"].astype(int)
cohort["min_signup_date"] = pd.to_datetime(cohort["min_signup_date"])

In [6]:
# cohort month and age of cohorts
cohort["cohort_month"] = cohort["min_signup_date"].dt.strftime("%Y-%m")
cohort["cohort_age"] = cohort["month_active"].apply(np.ceil)
# cohort["cohort_age"] = cohort["month_active"].astype(int)

# size of cohort per month
cohort_sizes = (
    cohort[cohort["cohort_age"] == 1]
    .groupby("cohort_month")["user_id"]
    .nunique()
    .reset_index(name="cohort_size")
)

# user per cohort age
retention = (
    cohort.groupby(["cohort_month", "cohort_age"])["user_id"]
    .nunique()
    .reset_index(name="num_users")
)

# calculate retention rate
retention = retention.merge(cohort_sizes, on="cohort_month")
retention["retention_rate"] = retention["num_users"] / retention["cohort_size"]

In [7]:
cohort.groupby(["cohort_age"])["user_id"].nunique()

cohort_age
1.0      1094
2.0     10545
3.0     22994
4.0     27858
5.0     26748
6.0     23696
7.0     21484
8.0     18034
9.0     13523
10.0     9400
11.0     6703
12.0     4046
13.0     1143
Name: user_id, dtype: int64

In [8]:
retention_pivot = retention.pivot_table(
    index="cohort_month", columns="cohort_age", values="retention_rate"
)

retention_pivot

cohort_age,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
cohort_month,,,,,,,,,,,,,
2022-07,1.0,16.000000,29.000000,51.000000,181.000000,517.000000,1356.000000,2343.000000,2735.0,2118.0,2533.0,3035.0,1143.0
2022-08,1.0,1.500000,11.500000,51.000000,201.500000,640.500000,1245.500000,1436.000000,1145.5,1359.0,1585.0,505.5,NaN
2022-10,1.0,26.000000,308.000000,1252.000000,2920.000000,3400.000000,3017.000000,3589.000000,4168.0,1226.0,NaN,NaN,NaN
2022-11,1.0,78.000000,569.000000,1554.000000,1918.500000,1697.500000,2145.000000,2533.500000,681.5,NaN,NaN,NaN,NaN
2022-12,1.0,22.800000,98.966667,133.200000,130.533333,160.966667,192.866667,55.966667,NaN,NaN,NaN,NaN,NaN
2023-01,1.0,16.680412,35.670103,39.041237,53.474227,62.144330,16.412371,NaN,NaN,NaN,NaN,NaN,NaN
2023-02,1.0,7.453782,12.810924,21.277311,27.369748,6.857143,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03,1.0,6.470833,19.141667,32.866667,10.470833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04,1.0,15.083333,49.091667,18.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(
    retention_pivot,
    cmap="YlGnBu",
    annot=True,
    fmt=".0%",
    cbar_kws={"label": "Retention rate"},
    linewidths=0.5,
)

plt.title("User Retention by Cohort Month")
plt.ylabel("Cohort Month")
plt.xlabel("Cohort Age (months)")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()